### __Load Environment Variables__

In [31]:
from dotenv import load_dotenv
load_dotenv()

True

### __State__

In [32]:
from typing import Annotated,Sequence, TypedDict

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages

class AgentState(TypedDict):
    """The state of the agent."""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    number_of_steps: int

### __Tools__

In [33]:
from langchain_tavily import TavilySearch
from langchain_core.tools import tool

@tool
def triple(num: float) -> float:
    """
    :param num: số cần được nhân ba
    :return: kết quả là số đó nhân với 3
    """
    return 3 * float(num)


tools = [TavilySearch(max_results=1), triple]

In [34]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini").bind_tools(tools)

### __Agent__

In [35]:
from langchain_core.messages import SystemMessage

system_prompt = SystemMessage(
    content="""
Bạn là một reasoning agent thông minh. Hãy luôn suy nghĩ từng bước một (step-by-step).

Hãy sử dụng định dạng sau:

Thought: suy nghĩ của bạn về việc cần làm tiếp theo (viết bằng Tiếng Việt)
Action: hành động cần thực hiện, ví dụ: `search`, `triple`
Action Input: đầu vào cho hành động đó
Observation: kết quả trả về của hành động

(Lặp lại quá trình Thought/Action/Observation nếu cần thiết)

Final Answer: câu trả lời cuối cùng gửi tới người dùng (viết bằng Tiếng Việt)

Question: {input}
"""
)

In [36]:
import json
from langchain_core.messages import ToolMessage
from langchain_core.runnables import RunnableConfig

tools_by_name = {tool.name: tool for tool in tools}


# Define tool node
def tool_node(state: AgentState):
    outputs = []
    for tool_call in state["messages"][-1].tool_calls:
        tool_result = tools_by_name[tool_call["name"]].invoke(tool_call["args"])
        outputs.append(
            ToolMessage(
                content=json.dumps(tool_result),
                name=tool_call["name"],
                tool_call_id=tool_call["id"],
            )
        )
    return {"messages": outputs}


# Define the node that calls the model
def call_model(
    state: AgentState,
    config: RunnableConfig,
):
    response = model.invoke([system_prompt] + state["messages"], config)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}


# Define the conditional edge that determines whether to continue or not
def should_continue(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

### __Graph__

In [37]:
from langgraph.graph import StateGraph, END

# Define a new graph
workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

# Set the entrypoint as "agent"
# This means that this node is the first one called
workflow.set_entry_point("agent")

# Add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use "agent".
    # This means these are the edges taken after the "agent" node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # Based on which one the output matches, that node will then be called.
    {
        # If "tools", then we call the tool node.
        "continue": "tools",
        # Otherwise we finish.
        "end": END,
    },
)

# We now add a normal edge from "tools" to "agent".
# This means that after "tools" is called, "agent" node is called next.
workflow.add_edge("tools", "agent")

# Now we can compile the graph
graph = workflow.compile()

In [38]:
graph.get_graph().draw_mermaid_png(output_file_path="static/react_agent_graph.png")
graph.get_graph().print_ascii()

        +-----------+         
        | __start__ |         
        +-----------+         
              *               
              *               
              *               
          +-------+           
          | agent |           
          +-------+           
         *         .          
       **           ..        
      *               .       
+-------+         +---------+ 
| tools |         | __end__ | 
+-------+         +---------+ 


### __Test__

In [39]:
inputs = {"messages": [("user", "Thời tiết ở Việt Nam như thế nào?")]}

for state in graph.stream(inputs, stream_mode="values"):
    last_message = state["messages"][-1]
    last_message.pretty_print()

================================ Human Message =================================

Thời tiết ở Việt Nam như thế nào?


================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_QOUCjuNbHCKPpL1ms5WniRUx)
 Call ID: call_QOUCjuNbHCKPpL1ms5WniRUx
  Args:
    query: Thời tiết ở Việt Nam
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "Th\u1eddi ti\u1ebft \u1edf Vi\u1ec7t Nam", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.accuweather.com/vi/vn/vietnam-weather", "title": "Th\u1eddi ti\u1ebft hi\u1ec7n t\u1ea1i tr\u00ean to\u00e0n qu\u1ed1c - AccuWeather", "content": "Vi\u1ec7t Nam \u0110i\u1ec1u ki\u1ec7n th\u1eddi ti\u1ebft Xem th\u00eam \u00b7 Ba V\u00ec 67\u00b0 B\u1eafc T\u1eeb Li\u00eam 69\u00b0 Bi\u00ean H\u00f2a 83\u00b0 B\u00ecnh Ch\u00e1nh 82\u00b0 B\u00ecnh T\u00e2n 82\u00b0 B\u00ecnh Th\u1ea1nh 82\u00b0 C\u1ea7n Th\u01a1 79\u00b0 C\u1ea7u Gi\u1ea5y 69\u00b0 Ch\u01b0\u01a1ng M\u1ef9", "score": 0.7730283, "raw_c

In [40]:
inputs = {"messages": [("user", "Thời tiết ở Việt Nam như thế nào? Sau đó nhân ba lên.")]}

for state in graph.stream(inputs, stream_mode="values"):
    last_message = state["messages"][-1]
    last_message.pretty_print()

================================ Human Message =================================

Thời tiết ở Việt Nam như thế nào? Sau đó nhân ba lên.


================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_O5IAYux2OqTxoqpURm75zrtp)
 Call ID: call_O5IAYux2OqTxoqpURm75zrtp
  Args:
    query: thời tiết ở Việt Nam
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "th\u1eddi ti\u1ebft \u1edf Vi\u1ec7t Nam", "response_time": 0.85, "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.accuweather.com/vi/vn/vietnam-weather", "title": "Th\u1eddi ti\u1ebft hi\u1ec7n t\u1ea1i tr\u00ean to\u00e0n qu\u1ed1c - AccuWeather", "content": "Vi\u1ec7t Nam \u0110i\u1ec1u ki\u1ec7n th\u1eddi ti\u1ebft Xem th\u00eam \u00b7 Ba V\u00ec 67\u00b0 B\u1eafc T\u1eeb Li\u00eam 69\u00b0 Bi\u00ean H\u00f2a 83\u00b0 B\u00ecnh Ch\u00e1nh 82\u00b0 B\u00ecnh T\u00e2n 82\u00b0 B\u00ecnh Th\u1ea1nh 82\u00b0 C\u1ea7n Th\u01a1 79\u00b0 C\u1ea7u Gi\u1ea5y 69\u00b0 Ch\u01b0\u01a1ng M\u1ef9", "score":